# Processamento de Linguagem Natural - Faculdade Descomplica
# Aula 10 - Linguagem Natural com NLTK

Prof. Elvis de Souza

# Baixando e importando NLTK e corpus

In [ ]:
! pip3 install nltk

In [ ]:
import nltk
nltk.download("machado")

In [ ]:
print(nltk.corpus.machado.readme())

In [ ]:
corpus = nltk.corpus.machado.raw("romance/marm08.txt")

In [ ]:
type(corpus)

In [ ]:
print(corpus[:1000])

# Pré-processando o corpus

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download("punkt")

In [ ]:
tokens = word_tokenize(corpus)

In [ ]:
print(tokens)

In [ ]:
type(tokens)

In [ ]:
tokens.count("Casmurro")

In [ ]:
tokens.count("Capitu")

# Explorando os personagens do corpus

In [ ]:
personagens = "Bentinho,Santiago,Casmurro,Capitu,Escobar,Glória,José,Cosme,Justina"
personagens = personagens.split(",")
personagens

In [ ]:
for personagem in personagens:
    print(personagem, ":", tokens.count(personagem))

# Usando o objeto Text do NLTK

In [ ]:
texto = nltk.Text(tokens)

In [ ]:
type(texto)

In [ ]:
texto.concordance("Capitu", width=120)

In [ ]:
texto.similar("Capitu")

In [ ]:
nltk.download("stopwords")

In [ ]:
texto.collocations(40)

In [ ]:
texto.dispersion_plot(["Capitu"])

In [ ]:
texto.dispersion_plot(personagens)

# Explorando o dicionário de frequências do NLTK

In [ ]:
freq = nltk.FreqDist(tokens)

In [ ]:
freq

In [ ]:
freq.plot(20)

# Removendo stopwords e normalizando

In [ ]:
import string

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [ ]:
print(stopwords)

In [ ]:
string.punctuation

In [ ]:
tokens_sem_stopwords = [x for x in tokens if x.lower() not in stopwords and x not in string.punctuation]

In [ ]:
freq_sem_stopwords = nltk.FreqDist(tokens_sem_stopwords)

In [ ]:
freq_sem_stopwords

In [ ]:
freq_sem_stopwords.plot(20)

# Vídeo 3: Treinando um modelo de classificação usando NLTK

In [ ]:
nltk.download("mac_morpho")

In [ ]:
print(nltk.corpus.mac_morpho.readme())

# Explorando o dataset MacMorpho

In [ ]:
frases_anotadas = nltk.corpus.mac_morpho.tagged_sents()

In [ ]:
frases_anotadas[0]

In [ ]:
len(frases_anotadas)

In [ ]:
tokens_anotados = nltk.corpus.mac_morpho.tagged_words()

In [ ]:
len(tokens_anotados)

In [ ]:
tokens_anotados[0]

In [ ]:
tokens_anotados[0][0]

In [ ]:
tokens_anotados[0][0].title()

In [ ]:
tokens_anotados[0][0] == tokens_anotados[0][0].title()

In [ ]:
tokens_anotados[1]

In [ ]:
tokens_anotados[1][0][-3]

In [ ]:
tokens_anotados[1][0][-3:]

In [ ]:
"Viagem".endswith("m")

In [ ]:
"Viagem".endswith("im")

# Calculando os sufixos mais frequentes das palavras

In [ ]:
freq_sufixos = nltk.FreqDist()
for token in tokens_anotados:
    palavra = token[0]

    ultima_letra = palavra[-1:].lower()
    duas_ultimas = palavra[-2:].lower()
    tres_ultimas = palavra[-3:].lower()

    freq_sufixos[ultima_letra] += 1
    if duas_ultimas != ultima_letra:
        freq_sufixos[duas_ultimas] += 1
        if tres_ultimas != duas_ultimas:
            freq_sufixos[tres_ultimas] += 1

In [ ]:
freq_sufixos

In [ ]:
sufixos_mais_comuns = [x[0] for x in freq_sufixos.most_common(100)]

In [ ]:
print(sufixos_mais_comuns)

# Criando função para extrair features das palavras

In [ ]:
def extrair_features(token):
    features = {}
    features["inicial_maiúscula"] = token == token.title()
    for sufixo in sufixos_mais_comuns:
        features["sufixo_" + sufixo] = token.lower().endswith(sufixo)
    return features

In [ ]:
extrair_features("Venham")

# Criando dataset de treino e de teste

In [ ]:
len(tokens_anotados)

In [ ]:
# Reduzindo tokens para poupar memória RAM
conjunto_features = []
for token, classe in tokens_anotados[:100000]:
    conjunto_features.append((extrair_features(token), classe))

In [ ]:
print(conjunto_features[0])

In [ ]:
print(conjunto_features[1])

In [ ]:
len(conjunto_features)

In [ ]:
tamanho_teste = int(len(conjunto_features) * 0.1)
tamanho_teste

In [ ]:
conjunto_teste = conjunto_features[:tamanho_teste]
conjunto_treino = conjunto_features[tamanho_teste:]

In [ ]:
len(conjunto_treino)

In [ ]:
len(conjunto_teste)

# Treinando o classificador

In [ ]:
classificador = nltk.NaiveBayesClassifier.train(conjunto_treino)

In [ ]:
classificador.classify(extrair_features("venham"))

In [ ]:
classificador.classify(extrair_features("Venham"))

In [ ]:
# ATENÇÃO: Demora alguns minutos
nltk.classify.accuracy(classificador, conjunto_teste)

# Vídeo 4: Avaliando um modelo de classificação usando NLTK

# Entendendo os erros do classificador

In [ ]:
tokens_anotados[0]

In [ ]:
for token in tokens_anotados[:50]:
    palavra = token[0]
    classe = token[1]
    previsao = classificador.classify(extrair_features(palavra))
    if classe != previsao:
        print((palavra, classe, previsao))

# Melhorando a função de extração de features das palavras, inserindo contexto para as palavras (palavras à esquerda e à direita)

In [ ]:
def extrair_features_contextuais(frase_tokenizada):
    features_da_frase = []
    for i, token in enumerate(frase_tokenizada):
        features = {}
        features["inicial_maiúscula"] = token == token.title()
        for sufixo in sufixos_mais_comuns:
            features["sufixo_" + sufixo] = token.lower().endswith(sufixo)

        if i == 0:
            token_anterior = "<INÍCIO>"
        else:
            token_anterior = frase_tokenizada[i-1]
        features["anterior_maiúscula"] = token_anterior == token_anterior.title()
        for sufixo in sufixos_mais_comuns:
            features["anterior_" + sufixo] = token_anterior.lower().endswith(sufixo)

        if (i+1) == len(frase_tokenizada):
            token_proximo = "<FIM>"
        else:
            token_proximo = frase_tokenizada[i+1]
        features["proximo_maiusculo"] = token_proximo == token_proximo.title()
        for sufixo in sufixos_mais_comuns:
            features["proximo_" + sufixo] = token_proximo.lower().endswith(sufixo)

        features_da_frase.append(features)
    return features_da_frase

# Criando dataset de treino e teste

In [ ]:
frases_anotadas[0]

In [ ]:
len(frases_anotadas)

In [ ]:
# Reduzindo frases para poupar RAM
frases_sem_anotacao = []
etiquetas_apenas = []
for frase in frases_anotadas[:5000]:
    etiquetas = []
    tokens = []
    for token in frase:
        tokens.append(token[0])
        etiquetas.append(token[1])
    frases_sem_anotacao.append(tokens)
    etiquetas_apenas.append(etiquetas)

In [ ]:
frases_sem_anotacao[0]

In [ ]:
etiquetas_apenas[0]

In [ ]:
extrair_features_contextuais(frases_sem_anotacao[0])

In [ ]:
conjunto_features = []
for i, frase in enumerate(frases_sem_anotacao):
    features_da_frase = extrair_features_contextuais(frase)
    for t, features_do_token in enumerate(features_da_frase):
        conjunto_features.append((features_do_token, etiquetas_apenas[i][t]))

In [ ]:
print(conjunto_features[0])

In [ ]:
print(conjunto_features[1])

In [ ]:
len(conjunto_features)

In [ ]:
tamanho_teste = int(len(conjunto_features) * 0.1)
tamanho_teste

In [ ]:
conjunto_teste = conjunto_features[:tamanho_teste]
conjunto_treino = conjunto_features[tamanho_teste:]

In [ ]:
len(conjunto_treino)

In [ ]:
len(conjunto_teste)

# Treinando o classificador

In [ ]:
classificador_contextual = nltk.NaiveBayesClassifier.train(conjunto_treino)

In [ ]:
extrair_features_contextuais(["venham", "ver", "o", "sol", "!"])

In [ ]:
frase1 = ["venham", "ver", "o", "sol", "!"]

In [ ]:
frase2 = ["Venham", "ver", "o", "Sol", "!"]

In [ ]:
for frase in [frase1, frase2]:
    features = extrair_features_contextuais(frase)
    for i, token in enumerate(features):
        classificacao = classificador_contextual.classify(token)
        print(frase[i], classificacao)

In [ ]:
# ATENÇÃO: Demora 4 minutos
nltk.classify.accuracy(classificador_contextual, conjunto_teste)